In [1]:
%load_ext autoreload
%autoreload 2

import sys, os, shutil
import pandas as pd
import numpy as np
from tqdm import tqdm
import wandb
import shutil

sys.path.append(os.path.join(os.getcwd(), '../symlie'))
from misc.utils import NumpyUtils, Results

In [2]:
log_dir = '../logs'
store_dir = os.path.join(log_dir, 'store')
wandb_dir = os.path.join(log_dir, 'wandb')

In [3]:
results_df_old = pd.read_pickle('../logs/store/results_df.pkl')
print(results_df_old.shape)
print(results_df_old.columns)
results_df_old.head()

FileNotFoundError: [Errno 2] No such file or directory: '../logs/store/results_df.pkl'

In [13]:
api = wandb.Api()
runs = api.runs('eliasdubbeldam/symlie')
run_id_api = {run.id : {'tags':run.tags, 'name':run.name} for run in runs}
print(len(run_id_api))
run_id_api

191


{'q3ic50qj': {'tags': ['long2', 'mnist'], 'name': 'divine-spaceship-418'},
 'i1dkddgf': {'tags': ['long2', 'mnist'], 'name': 'cosmic-universe-419'},
 'up6r9ere': {'tags': ['long2', 'mnist'], 'name': 'ancient-resonance-417'},
 'uu5nppu8': {'tags': ['dev'], 'name': 'legendary-music-410'},
 'pdfkzid7': {'tags': ['long', 'mnist-predict'], 'name': 'scarlet-sky-409'},
 'kitezbza': {'tags': ['long', 'mnist-predict'], 'name': 'likely-brook-407'},
 'u6cw81pd': {'tags': ['long', 'mnist-predict'], 'name': 'stellar-deluge-408'},
 'd1rljask': {'tags': ['long', 'mnist-predict'], 'name': 'icy-haze-406'},
 '87gwickx': {'tags': ['long', 'mnist-predict'], 'name': 'decent-sponge-405'},
 '8qbnp9sn': {'tags': ['long', 'mnist-predict'], 'name': 'cosmic-pine-404'},
 'ytp0ix8i': {'tags': ['dev', 'hparams', 'long', 'mnist'],
  'name': 'bumbling-jazz-403'},
 'fmxhxywt': {'tags': ['dev', 'hparams', 'long', 'mnist'],
  'name': 'graceful-puddle-401'},
 'qd8q06l7': {'tags': ['dev', 'hparams', 'long', 'mnist'],
  'n

In [33]:
tidy_dirs = {
    'P': {
        'dir'     : os.path.join(store_dir, 'P'), 
        'listdir' : lambda dir: [(dir, dir.split('.')[0]) for dir in os.listdir(dir)],
        'rmdir'   : lambda dir: os.remove(dir),
    },
    'symlie' : {
        'dir'     : os.path.join(log_dir, 'symlie'), 
        'listdir' : lambda dir: [(dir, dir) for dir in os.listdir(dir)],
        'rmdir'   : lambda dir: shutil.rmtree(dir),
    },
    'wandb' : {
        'dir'     : os.path.join(wandb_dir), 
        'listdir' : lambda dir: [(dir, dir.split('-')[-1]) for dir in os.listdir(dir) if dir.startswith('run')],
        'rmdir'   : lambda dir: shutil.rmtree(dir),
    },
}

for tidy_dir, t in tidy_dirs.items():
    for dir_full, dir_strip in t['listdir'](t['dir']):
        if dir_strip not in run_id_api.keys():
            print(f'Deleting {dir_full}')
            t['rmdir'](os.path.join(t['dir'], dir_full))

In [14]:
def exceptions(runs):
    runs_new = []
    for run in runs:

        if (run.state != 'finished'): continue
        # if id in results_df_old['run_id'].values: continue 
        if 'dev' in run.tags: continue
        
        print(run.id)
        runs_new.append(run)
    return runs_new

runs_selected = exceptions(runs)
print(len(runs_selected))

q3ic50qj
i1dkddgf
up6r9ere
pdfkzid7
kitezbza
u6cw81pd
d1rljask
87gwickx
8qbnp9sn
2zvdscih
np2cw13y
s7qyfq0o
ewvypada
4o6s66dg
cpu6bmf2
9ecj907t
s1l80nqk
ghjre4n4
vqc1ga7d
zoeqac33
bo0u6rro
5xraaffx
22d5kkqb
0t0e1cqn
uaxhdpz7
rex8xr0p
q3m8184p
1ousktd6
9nwnzq28
eznwkfdg
92j55rgo
ety3ctmi
gcqu51uq
f74bjwl2
x33xeio3
l5cm7g1j
oynnamyg
7akg7ozg
tv1xv5f3
ls3mo83a
b3hzpwd9
wam9esyd
hv1qjz16
2lzvceyl
kcisvcap
ksz69o5j
in69mwkj
42b5jtma
mvpwp51a
iugjxsv4
d8e2f8th
pl22gfan
ms59mg2y
vx9dm5xr
rs4qh5fj
5s4w4gm5
4f5mk9q1
7d44epbn
g3mirzzk
b2tuqy7n
4tt1qogc
chk3d4pf
bzv9x8m1
zewfef4f
03sba2gu
4bxjjhy5
miulsn4h
uegf6m2h
vz4jfzh6
wqmfolq3
5k72ln34
2prolhx1
mg3t7tf9
nxegq9to
w3n48isl
wju0ax0z
9g2e5b7x
3sfu5vao
cv0xmbsk
jnl5htnv
d5xoeic9
ji17zttk
zh8oziq9
fg592l6n
0p6wxtxq
vgc5pdak
6e46ouhg
8il970oo
xossks2f
2i4yegut
7i3ucxpz
xlzs1sob
whgbybzs
chzrf7jq
788kujfo
ihexez6d
jqitwonj
cd5vdvxh
rq0day54
i1l5ulvv
cvph82t5
nn3p70xp
76npyvww
xppfwtlj
69ennlv5
ckwjzr8c
tz7ndvlp
2m7eu5tn
ugylvzz6
unpw27ml
w4yjcu25
q

In [15]:
config_list = []

pbar = tqdm(runs_selected)
for run in pbar:
    id = run.id
    pbar.set_description(f'Retreiving wandb {str(id)}')

    # Retreive config from wandb run, add all info to config
    config = run.config

    # Add run id and name to config
    config['run_id'] = id
    config['run_name'] = run.name

    # Test loss
    try:
        test_loss_history = run.history(keys=['test_loss'])['test_loss']
        if len(test_loss_history) != 1:
            print(f'Warning: {id} has test_loss {test_loss_history}')
        # test_losses = np.array(history['test_loss'].dropna())
        # print(run.id, test_losses)
        config['test_loss'] = test_loss_history.item()
    except:
        config['test_loss'] = np.nan

    config_list.append(config)

results_df = pd.DataFrame(config_list)
results_df

Retreiving wandb xlqnfg5w: 100%|██████████| 147/147 [00:32<00:00,  4.49it/s]


,lr,net,bias,name,seed,tags,test,n_val,train,y_low,...,num_workers,generate_data,model_summary,args_processed,do_return_model,transform_kwargs,persistent_workers,run_name,test_loss,space_length
0,0.0100,TrainP,False,None,3,"[mnist, long2]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,divine-spaceship-418,0.000711,NaN
1,0.0100,TrainP,False,None,1,"[mnist, long2]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,cosmic-universe-419,0.000709,NaN
2,0.0100,TrainP,False,None,2,"[mnist, long2]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,ancient-resonance-417,0.000752,NaN
3,0.0010,Predict-TrainedP,False,None,3,"[mnist-predict, long]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,scarlet-sky-409,2.168962,NaN
4,0.0010,Predict-TrainedP,False,None,1,"[mnist-predict, long]",True,1000,True,NaN,...,18,False,False,True,False,"{'eps_mult': [0, 0, 1, 1], 'only_flip': False}",True,likely-brook-407,1.779646,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,0.0100,TrainP,False,None,3,[v5],True,100,True,1.0,...,18,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,prime-aardvark-151,0.023908,7.0
143,0.0100,TrainP,False,None,2,[v5],True,100,True,1.0,...,18,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,young-wind-158,0.003005,7.0
144,0.0001,TrainP,False,None,2,[v5],True,100,True,1.0,...,18,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,eternal-pine-146,0.009467,7.0
145,0.0010,TrainP,False,None,1,[v5],True,100,True,1.0,...,18,False,False,True,False,"{'augment': 'transform_flower', 'eps_mult': [0...",True,fragrant-plasma-147,0.000438,7.0


In [ ]:
results_df.to_pickle(os.path.join(store_dir, 'results_df.pkl'))

# results_df_new = pd.concat([results_df_old, results_df]).reset_index(drop=True)
# results_df_new.to_pickle(os.path.join(store_dir, 'results_df.pkl'))